In [20]:
# modules
import numpy as np
import pandas as pd
from pathlib import Path
import os
from lightgbm import LGBMClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score, accuracy_score

In [21]:
# change dir
%cd /home/hirano/work/Competition/Nishika_cable/src/
%load_ext autoreload
%autoreload 2
!pwd

/home/hirano/work/Competition/Nishika_cable/src
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/hirano/work/Competition/Nishika_cable/src


In [22]:
os.makedirs('../output/ensemble', exist_ok=True)

pref = '15' #!!Please change Submit Number !!

#versions = ['010', '011', '006']

# Do All

In [23]:
import itertools
lis=['006', '007', '008', '009', '010', '030', '035', '037', '039', '043', '044', '045', '046']
result = []
for n in range(1,len(lis)+1):
    for conb in itertools.combinations(lis, n):
        result.append(list(conb)) #タプルをリスト型に変換
print(len(result))

8191


In [24]:
params = {
    'objective': 'multiclass',
    'num_class' : 15,
    'metrics': 'multi_error',
    'n_estimators': 10000,
    'boosting_type': 'gbdt',
    'num_leaves': 32,
    'max_depth': 2,
    'learning_rate': 0.01,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.3,
    'bagging_freq': 5,
    'reg_alpha': 0.5,
    'reg_lambda': 0,
}

In [27]:
best_scores

[[0.9855685493903956, ['006', '007']],
 [0.9865638218462304, ['006', '044']],
 [0.9883055486439413, ['007', '037']]]

In [26]:


best = 0
best_scores = []

for ret in result:
    versions = ret
    if(2<=len(versions)<=3):
        
        # Load
        for i, version in enumerate(versions):
            oof_name = [x for x in os.listdir(f'../output/results/') if f'oof_{version}' in x]
            #print(oof_name, f'../output/{oof_name[0]}')

            if i==0:
                oof = pd.read_csv(f'../output/results/{oof_name[0]}').sort_values('filename')
                oof[f'{version}_pred_{i}'] = oof.pred.values
                oof[[f'{version}_conf_0', f'{version}_conf_1', f'{version}_conf_2', f'{version}_conf_3', f'{version}_conf_4',
                     f'{version}_conf_5', f'{version}_conf_6', f'{version}_conf_7', f'{version}_conf_8', f'{version}_conf_9',
                     f'{version}_conf_10', f'{version}_conf_11', f'{version}_conf_12', f'{version}_conf_13', f'{version}_conf_14']] = oof[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4','conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9','conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']].values
                

                sub = pd.read_csv(f'../output/results/sub_{version}.csv')
                sub[f'{version}_pred_{i}'] = sub.target.values
                sub[[f'{version}_conf_0', f'{version}_conf_1', f'{version}_conf_2', f'{version}_conf_3', f'{version}_conf_4',
                     f'{version}_conf_5', f'{version}_conf_6', f'{version}_conf_7', f'{version}_conf_8', f'{version}_conf_9',
                     f'{version}_conf_10', f'{version}_conf_11', f'{version}_conf_12', f'{version}_conf_13', f'{version}_conf_14']] = sub[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4','conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9','conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']].values
                
                oof[f'{version}_conf_4'] = oof[f'{version}_conf_4']*0.9
                sub[f'{version}_conf_4'] = sub[f'{version}_conf_4']*0.9

            else:
                _tmp = pd.read_csv(f'../output/results/{oof_name[0]}').sort_values('filename')
                oof[f'{version}_pred_{i}'] = _tmp.pred.values
                oof[[f'{version}_conf_0', f'{version}_conf_1', f'{version}_conf_2', f'{version}_conf_3', f'{version}_conf_4',
                     f'{version}_conf_5', f'{version}_conf_6', f'{version}_conf_7', f'{version}_conf_8', f'{version}_conf_9',
                     f'{version}_conf_10', f'{version}_conf_11', f'{version}_conf_12', f'{version}_conf_13', f'{version}_conf_14']] = _tmp[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4','conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9','conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']].values

                _tmp = pd.read_csv(f'../output/results/sub_{version}.csv')
                sub[f'{version}_pred_{i}'] = _tmp.target.values
                sub[[f'{version}_conf_0', f'{version}_conf_1', f'{version}_conf_2', f'{version}_conf_3', f'{version}_conf_4',
                     f'{version}_conf_5', f'{version}_conf_6', f'{version}_conf_7', f'{version}_conf_8', f'{version}_conf_9',
                     f'{version}_conf_10', f'{version}_conf_11', f'{version}_conf_12', f'{version}_conf_13', f'{version}_conf_14']] = _tmp[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4','conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9','conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']].values

                oof[f'{version}_conf_4'] = oof[f'{version}_conf_4']*0.9
                sub[f'{version}_conf_4'] = sub[f'{version}_conf_4']*0.9

                
        oof = oof.reset_index(drop=True)

        stacking_oof = np.zeros(len(oof))
        #stacking_sub = np.zeros(len(sub))
        stacking_sub = []

        #features = []
        features = [f'{versions[i]}_pred_{i}' for i in range(len(versions))]
        for version in versions:
            features.extend([f'{version}_conf_{i}' for i in range(15)])
        print(features)

        def lgb_f1_score(y_hat,data):
            y_true = data.get_label()
            y_hat = np.round(y_hat)
            return 'f1', f1_score(y_true, y_hat,average='macro'), True

        n_fold = 5

        for fold in range(n_fold):
            #print(f'{fold=}')

            trn = oof[oof.fold!=fold]
            val = oof[oof.fold==fold]

            val_idx = val.index

            trn_x = trn[features]
            trn_y = trn['target']
            val_x = val[features]
            val_y = val['target']

            tst_x = sub[features]

            model = LGBMClassifier(**params)
            model.fit(
                trn_x,
                trn_y,
                #eval_metric='multi_logloss',
                eval_set=[(val_x, val_y)],
                verbose=100,
                early_stopping_rounds=200
            )

            val_pred = model.predict(val_x)
            #stacking_sub += model.predict(tst_x)
            stacking_sub.append(model.predict(tst_x))

            stacking_oof[val_idx] = val_pred

        oof.pred = stacking_oof
        score = f1_score(oof.target.values, oof.pred.values, average='micro')
        print(f'Score is {score:.4f}', score, versions)

        if score > best:
            best = score
            print('**********************************************')
            print(score, versions)
            print('**********************************************')
            best_scores.append([score, versions])

        #break

['006_pred_0', '007_pred_1', '006_conf_0', '006_conf_1', '006_conf_2', '006_conf_3', '006_conf_4', '006_conf_5', '006_conf_6', '006_conf_7', '006_conf_8', '006_conf_9', '006_conf_10', '006_conf_11', '006_conf_12', '006_conf_13', '006_conf_14', '007_conf_0', '007_conf_1', '007_conf_2', '007_conf_3', '007_conf_4', '007_conf_5', '007_conf_6', '007_conf_7', '007_conf_8', '007_conf_9', '007_conf_10', '007_conf_11', '007_conf_12', '007_conf_13', '007_conf_14']
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.3, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3
Training until validation scores don't improve for 200 rounds
[100]	valid_0's multi_error: 0.00870647
[200]	valid_0's multi_error: 0.00995025
[300]	valid_0's multi_error: 0.00995025
Early stop

[100]	valid_0's multi_error: 0.0223881
[200]	valid_0's multi_error: 0.0199005
[300]	valid_0's multi_error: 0.0199005
Early stopping, best iteration is:
[151]	valid_0's multi_error: 0.0199005
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.3, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3
Training until validation scores don't improve for 200 rounds
[100]	valid_0's multi_error: 0.00746269
[200]	valid_0's multi_error: 0.00621891
[300]	valid_0's multi_error: 0.00621891
Early stopping, best iteration is:
[114]	valid_0's multi_error: 0.00621891
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. C

[100]	valid_0's multi_error: 0.0236613
[200]	valid_0's multi_error: 0.0249066
[300]	valid_0's multi_error: 0.0261519
Early stopping, best iteration is:
[131]	valid_0's multi_error: 0.0224159
Score is 0.9846 0.9845732769345609 ['006', '030']
['006_pred_0', '035_pred_1', '006_conf_0', '006_conf_1', '006_conf_2', '006_conf_3', '006_conf_4', '006_conf_5', '006_conf_6', '006_conf_7', '006_conf_8', '006_conf_9', '006_conf_10', '006_conf_11', '006_conf_12', '006_conf_13', '006_conf_14', '035_conf_0', '035_conf_1', '035_conf_2', '035_conf_3', '035_conf_4', '035_conf_5', '035_conf_6', '035_conf_7', '035_conf_8', '035_conf_9', '035_conf_10', '035_conf_11', '035_conf_12', '035_conf_13', '035_conf_14']
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.3, subsample=1.0 will

[100]	valid_0's multi_error: 0.0236318
[200]	valid_0's multi_error: 0.0223881
[300]	valid_0's multi_error: 0.0223881
[400]	valid_0's multi_error: 0.0199005
[500]	valid_0's multi_error: 0.0199005
Early stopping, best iteration is:
[349]	valid_0's multi_error: 0.0199005
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.3, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3
Training until validation scores don't improve for 200 rounds
[100]	valid_0's multi_error: 0.00870647
[200]	valid_0's multi_error: 0.00870647
Early stopping, best iteration is:
[30]	valid_0's multi_error: 0.00870647
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5

Early stopping, best iteration is:
[186]	valid_0's multi_error: 0.0174129
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.3, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3
Training until validation scores don't improve for 200 rounds
[100]	valid_0's multi_error: 0.0224159
[200]	valid_0's multi_error: 0.0199253
[300]	valid_0's multi_error: 0.0224159
Early stopping, best iteration is:
[138]	valid_0's multi_error: 0.0199253
Score is 0.9866 0.9865638218462304 ['006', '044']
**********************************************
0.9865638218462304 ['006', '044']
**********************************************
['006_pred_0', '045_pred_1', '006_conf_0', '006_conf_1', '006_conf_2', '006_conf_3', '006_conf_4', '006_conf_5', '006_conf_6', '006_conf_7', '006_

...

[100]	valid_0's multi_error: 0.00870647
[200]	valid_0's multi_error: 0.00870647
[300]	valid_0's multi_error: 0.00870647
Early stopping, best iteration is:
[150]	valid_0's multi_error: 0.00746269
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.3, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3
Training until validation scores don't improve for 200 rounds
[100]	valid_0's multi_error: 0.0174129
[200]	valid_0's multi_error: 0.0174129
Early stopping, best iteration is:
[53]	valid_0's multi_error: 0.0174129
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] 

[100]	valid_0's multi_error: 0.00870647
[200]	valid_0's multi_error: 0.00995025
Early stopping, best iteration is:
[79]	valid_0's multi_error: 0.00870647
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.3, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3
Training until validation scores don't improve for 200 rounds
[100]	valid_0's multi_error: 0.0161692
[200]	valid_0's multi_error: 0.0161692
Early stopping, best iteration is:
[42]	valid_0's multi_error: 0.0161692
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.3, su

[100]	valid_0's multi_error: 0.0199253
[200]	valid_0's multi_error: 0.0199253
Early stopping, best iteration is:
[59]	valid_0's multi_error: 0.0174346
Score is 0.9871 0.9870614580741478 ['039', '044', '045']
['039_pred_0', '044_pred_1', '046_pred_2', '039_conf_0', '039_conf_1', '039_conf_2', '039_conf_3', '039_conf_4', '039_conf_5', '039_conf_6', '039_conf_7', '039_conf_8', '039_conf_9', '039_conf_10', '039_conf_11', '039_conf_12', '039_conf_13', '039_conf_14', '044_conf_0', '044_conf_1', '044_conf_2', '044_conf_3', '044_conf_4', '044_conf_5', '044_conf_6', '044_conf_7', '044_conf_8', '044_conf_9', '044_conf_10', '044_conf_11', '044_conf_12', '044_conf_13', '044_conf_14', '046_conf_0', '046_conf_1', '046_conf_2', '046_conf_3', '046_conf_4', '046_conf_5', '046_conf_6', '046_conf_7', '046_conf_8', '046_conf_9', '046_conf_10', '046_conf_11', '046_conf_12', '046_conf_13', '046_conf_14']
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: f

[100]	valid_0's multi_error: 0.011194
[200]	valid_0's multi_error: 0.011194
Early stopping, best iteration is:
[98]	valid_0's multi_error: 0.011194
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.3, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3
Training until validation scores don't improve for 200 rounds
[100]	valid_0's multi_error: 0.0149254
[200]	valid_0's multi_error: 0.0149254
Early stopping, best iteration is:
[98]	valid_0's multi_error: 0.0149254
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.3, subsampl

[100]	valid_0's multi_error: 0.011194
[200]	valid_0's multi_error: 0.0124378
Early stopping, best iteration is:
[49]	valid_0's multi_error: 0.011194
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.3, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3
Training until validation scores don't improve for 200 rounds
[100]	valid_0's multi_error: 0.0186567
[200]	valid_0's multi_error: 0.0186567
Early stopping, best iteration is:
[38]	valid_0's multi_error: 0.0186567
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.3, subsamp

In [8]:
versions

['008', '010', '039', '043', '044', '045', '046']

In [7]:
best_scores

[[0.9838268225926847, ['006']],
 [0.986066185618313, ['007']],
 [0.9865638218462304, ['010']],
 [0.9870614580741478, ['037']],
 [0.9883055486439413, ['007', '037']],
 [0.9885543667579, ['006', '007', '030', '037']],
 [0.9888031848718587, ['007', '009', '037', '044']]]

In [14]:
best_scores

[[0.9865638218462304, ['010']],
 [0.9870614580741478, ['037']],
 [0.9880567305299826, ['010', '037']],
 [0.9883055486439413, ['010', '035', '044']],
 [0.9885543667579, ['010', '035', '044', '046']]]

# One

In [9]:
os.makedirs('../output/ensemble', exist_ok=True)

pref = '15' #!!Please change Submit Number !!

versions = ['010', '035', '044', '046']

In [10]:
# Load
for i, version in enumerate(versions):
    oof_name = [x for x in os.listdir(f'../output/results/') if f'oof_{version}' in x]
    #print(oof_name, f'../output/{oof_name[0]}')

    if i==0:
        oof = pd.read_csv(f'../output/results/{oof_name[0]}').sort_values('filename')
        oof[f'{version}_pred_{i}'] = oof.pred.values
        oof[[f'{version}_conf_0', f'{version}_conf_1', f'{version}_conf_2', f'{version}_conf_3', f'{version}_conf_4',
             f'{version}_conf_5', f'{version}_conf_6', f'{version}_conf_7', f'{version}_conf_8', f'{version}_conf_9',
             f'{version}_conf_10', f'{version}_conf_11', f'{version}_conf_12', f'{version}_conf_13', f'{version}_conf_14']] = oof[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4','conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9','conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']].values

        sub = pd.read_csv(f'../output/results/sub_{version}.csv')
        sub[f'{version}_pred_{i}'] = sub.target.values
        sub[[f'{version}_conf_0', f'{version}_conf_1', f'{version}_conf_2', f'{version}_conf_3', f'{version}_conf_4',
             f'{version}_conf_5', f'{version}_conf_6', f'{version}_conf_7', f'{version}_conf_8', f'{version}_conf_9',
             f'{version}_conf_10', f'{version}_conf_11', f'{version}_conf_12', f'{version}_conf_13', f'{version}_conf_14']] = sub[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4','conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9','conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']].values

    else:
        _tmp = pd.read_csv(f'../output/results/{oof_name[0]}').sort_values('filename')
        oof[f'{version}_pred_{i}'] = _tmp.pred.values
        oof[[f'{version}_conf_0', f'{version}_conf_1', f'{version}_conf_2', f'{version}_conf_3', f'{version}_conf_4',
             f'{version}_conf_5', f'{version}_conf_6', f'{version}_conf_7', f'{version}_conf_8', f'{version}_conf_9',
             f'{version}_conf_10', f'{version}_conf_11', f'{version}_conf_12', f'{version}_conf_13', f'{version}_conf_14']] = _tmp[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4','conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9','conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']].values

        _tmp = pd.read_csv(f'../output/results/sub_{version}.csv')
        sub[f'{version}_pred_{i}'] = _tmp.target.values
        sub[[f'{version}_conf_0', f'{version}_conf_1', f'{version}_conf_2', f'{version}_conf_3', f'{version}_conf_4',
             f'{version}_conf_5', f'{version}_conf_6', f'{version}_conf_7', f'{version}_conf_8', f'{version}_conf_9',
             f'{version}_conf_10', f'{version}_conf_11', f'{version}_conf_12', f'{version}_conf_13', f'{version}_conf_14']] = _tmp[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4','conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9','conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']].values
#
oof = oof.reset_index(drop=True)

In [11]:
oof.head()

,filename,Xmin_Ymin_Xmax_Ymax,class,filename_trim,fold,target,pred,conf_0,conf_1,conf_2,...,046_conf_5,046_conf_6,046_conf_7,046_conf_8,046_conf_9,046_conf_10,046_conf_11,046_conf_12,046_conf_13,046_conf_14
0,00037f39.jpg,NaN,Lightning,00037f39_trim.jpg,1,3,3,0.001592,0.001545,0.000401,...,0.000191,0.000282,0.000292,0.005821,0.000022,0.000078,0.000565,0.003767,0.000036,0.000040
1,000c15d0.jpg,"[906, 1686, 1294, 1946]",USB_Type_C,NaN,2,8,8,0.000120,0.000084,0.000268,...,0.001538,0.000085,0.001652,0.661621,0.000265,0.000136,0.000262,0.012169,0.000199,0.000279
2,000f21ac.jpg,NaN,USB_Type_C,000f21ac_trim.jpg,3,8,8,0.000426,0.000364,0.001211,...,0.000089,0.000068,0.000863,0.926270,0.000087,0.000029,0.000878,0.002333,0.000045,0.000062
3,0013fa17.jpg,"[1458, 1756, 2304, 2602]",DisplayPort,NaN,0,14,14,0.000330,0.000319,0.000185,...,0.005806,0.003380,0.361328,0.024719,0.007137,0.003742,0.005386,0.001007,0.012177,0.395264
4,00150bc1.jpg,"[1181, 1692, 1658, 1985]",USB_Type_C,NaN,2,8,8,0.000223,0.000254,0.000327,...,0.000362,0.000434,0.001311,0.760742,0.000381,0.000075,0.000198,0.003374,0.000202,0.005066


In [12]:
oof.head()

,filename,Xmin_Ymin_Xmax_Ymax,class,filename_trim,fold,target,pred,conf_0,conf_1,conf_2,...,046_conf_5,046_conf_6,046_conf_7,046_conf_8,046_conf_9,046_conf_10,046_conf_11,046_conf_12,046_conf_13,046_conf_14
0,00037f39.jpg,NaN,Lightning,00037f39_trim.jpg,1,3,3,0.001592,0.001545,0.000401,...,0.000191,0.000282,0.000292,0.005821,0.000022,0.000078,0.000565,0.003767,0.000036,0.000040
1,000c15d0.jpg,"[906, 1686, 1294, 1946]",USB_Type_C,NaN,2,8,8,0.000120,0.000084,0.000268,...,0.001538,0.000085,0.001652,0.661621,0.000265,0.000136,0.000262,0.012169,0.000199,0.000279
2,000f21ac.jpg,NaN,USB_Type_C,000f21ac_trim.jpg,3,8,8,0.000426,0.000364,0.001211,...,0.000089,0.000068,0.000863,0.926270,0.000087,0.000029,0.000878,0.002333,0.000045,0.000062
3,0013fa17.jpg,"[1458, 1756, 2304, 2602]",DisplayPort,NaN,0,14,14,0.000330,0.000319,0.000185,...,0.005806,0.003380,0.361328,0.024719,0.007137,0.003742,0.005386,0.001007,0.012177,0.395264
4,00150bc1.jpg,"[1181, 1692, 1658, 1985]",USB_Type_C,NaN,2,8,8,0.000223,0.000254,0.000327,...,0.000362,0.000434,0.001311,0.760742,0.000381,0.000075,0.000198,0.003374,0.000202,0.005066


In [13]:
sub.head()

,target,conf_0,conf_1,conf_2,conf_3,conf_4,conf_5,conf_6,conf_7,conf_8,...,046_conf_5,046_conf_6,046_conf_7,046_conf_8,046_conf_9,046_conf_10,046_conf_11,046_conf_12,046_conf_13,046_conf_14
0,8,0.003019,0.003868,0.003102,0.001565,0.008576,0.000612,0.001306,0.002652,0.963867,...,0.008675,0.000119,0.000849,0.736328,0.000462,0.000139,0.000873,0.010441,0.000229,0.000543
1,4,0.000227,0.007034,0.000111,0.002904,0.982617,0.000979,0.000262,0.001295,0.000922,...,0.005446,0.000024,0.000178,0.001232,0.000208,0.000062,0.001006,0.001266,0.000014,0.000115
2,8,0.000309,0.000845,0.000094,0.003567,0.021788,0.000092,0.000033,0.002581,0.969043,...,0.000517,0.000088,0.001280,0.204590,0.000135,0.000536,0.000508,0.007954,0.000082,0.000364
3,4,0.001772,0.001058,0.000670,0.010768,0.916699,0.000991,0.000484,0.002420,0.022867,...,0.003494,0.000065,0.000654,0.960840,0.000107,0.000159,0.000128,0.009383,0.000025,0.000692
4,1,0.001641,0.972656,0.000084,0.003753,0.010972,0.001489,0.000213,0.002646,0.005512,...,0.012177,0.000264,0.002099,0.076865,0.000545,0.002966,0.001920,0.018738,0.000116,0.000524


In [14]:
params = {
    'objective': 'multiclass',
    'num_class' : 15,
    'metrics': 'multi_error',
    'n_estimators': 10000,
    'boosting_type': 'gbdt',
    'num_leaves': 32,
    'max_depth': 2,
    'learning_rate': 0.01,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.3,
    'bagging_freq': 5,
    'reg_alpha': 0.5,
    'reg_lambda': 0,
}

In [15]:
val[features]

,008_pred_0,010_pred_1,039_pred_2,043_pred_3,044_pred_4,045_pred_5,046_pred_6,008_conf_0,008_conf_1,008_conf_2,...,046_conf_5,046_conf_6,046_conf_7,046_conf_8,046_conf_9,046_conf_10,046_conf_11,046_conf_12,046_conf_13,046_conf_14
2,8,8,8,8,8,8,8,0.000170,0.000282,0.000375,...,0.000089,0.000068,0.000863,0.926270,0.000087,0.000029,0.000878,0.002333,0.000045,0.000062
9,8,8,8,8,8,8,8,0.000227,0.000545,0.000344,...,0.034821,0.000379,0.013580,0.600098,0.000869,0.007835,0.001783,0.009117,0.000831,0.000915
28,4,4,4,4,4,4,4,0.000671,0.002914,0.000143,...,0.010941,0.001040,0.000680,0.139404,0.003004,0.000927,0.000554,0.006123,0.002491,0.001794
30,14,14,14,14,14,14,14,0.000268,0.000316,0.000170,...,0.001474,0.001518,0.002007,0.001156,0.001634,0.000683,0.000491,0.000140,0.000495,0.986816
32,4,4,4,4,4,4,4,0.000042,0.000411,0.000039,...,0.000083,0.000009,0.000105,0.001096,0.000429,0.000034,0.000392,0.000633,0.000005,0.000034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4005,7,7,7,7,7,7,7,0.000226,0.000552,0.000212,...,0.001334,0.002237,0.912598,0.043854,0.000359,0.007282,0.001479,0.000310,0.001989,0.000467
4008,8,8,8,8,8,8,8,0.000191,0.000176,0.000139,...,0.000522,0.000317,0.004040,0.573730,0.000122,0.000485,0.001352,0.052429,0.000481,0.000354
4011,6,6,6,6,6,6,6,0.000434,0.000773,0.000301,...,0.001263,0.875000,0.000807,0.000933,0.030472,0.002621,0.002764,0.008507,0.000760,0.038361
4017,3,3,3,3,3,3,3,0.000213,0.000204,0.000088,...,0.000136,0.000426,0.000817,0.042419,0.000028,0.000098,0.001817,0.000678,0.000176,0.000034


In [16]:
stacking_oof = np.zeros(len(oof))
#stacking_sub = np.zeros(len(sub))
stacking_sub = []

#features = []
features = [f'{versions[i]}_pred_{i}' for i in range(len(versions))]
for version in versions:
    features.extend([f'{version}_conf_{i}' for i in range(15)])
print(features)

def lgb_f1_score(y_hat,data):
    y_true = data.get_label()
    y_hat = np.round(y_hat)
    return 'f1', f1_score(y_true, y_hat,average='macro'), True

n_fold = 5

for fold in range(n_fold):
    print(f'{fold=}')

    trn = oof[oof.fold!=fold]
    val = oof[oof.fold==fold]

    val_idx = val.index

    trn_x = trn[features]
    trn_y = trn['target']
    val_x = val[features]
    val_y = val['target']

    tst_x = sub[features]

    model = LGBMClassifier(**params)
    model.fit(
        trn_x,
        trn_y,
        #eval_metric='multi_logloss',
        eval_set=[(val_x, val_y)],
        verbose=100,
        early_stopping_rounds=200
    )
    
    val_pred = model.predict(val_x)
    #stacking_sub += model.predict(tst_x)
    stacking_sub.append(model.predict(tst_x))
    
    stacking_oof[val_idx] = val_pred
    #break

['010_pred_0', '035_pred_1', '044_pred_2', '046_pred_3', '010_conf_0', '010_conf_1', '010_conf_2', '010_conf_3', '010_conf_4', '010_conf_5', '010_conf_6', '010_conf_7', '010_conf_8', '010_conf_9', '010_conf_10', '010_conf_11', '010_conf_12', '010_conf_13', '010_conf_14', '035_conf_0', '035_conf_1', '035_conf_2', '035_conf_3', '035_conf_4', '035_conf_5', '035_conf_6', '035_conf_7', '035_conf_8', '035_conf_9', '035_conf_10', '035_conf_11', '035_conf_12', '035_conf_13', '035_conf_14', '044_conf_0', '044_conf_1', '044_conf_2', '044_conf_3', '044_conf_4', '044_conf_5', '044_conf_6', '044_conf_7', '044_conf_8', '044_conf_9', '044_conf_10', '044_conf_11', '044_conf_12', '044_conf_13', '044_conf_14', '046_conf_0', '046_conf_1', '046_conf_2', '046_conf_3', '046_conf_4', '046_conf_5', '046_conf_6', '046_conf_7', '046_conf_8', '046_conf_9', '046_conf_10', '046_conf_11', '046_conf_12', '046_conf_13', '046_conf_14']
fold=0
Training until validation scores don't improve for 200 rounds
[100]	valid_0'

# OOF

In [17]:
oof.pred = stacking_oof
print(oof.pred)

score = f1_score(oof.target.values, oof.pred.values, average='micro')
print(f'Score is {score:.8f}')

sub_name = '_'.join(versions)
oof.to_csv(f'../output/ensemble/{pref}_oof_stacking_{sub_name}_{score:.4f}.csv', index=False)

0        3.0
1        8.0
2        8.0
3       14.0
4        8.0
        ... 
4014     4.0
4015     3.0
4016     6.0
4017     3.0
4018     4.0
Name: pred, Length: 4019, dtype: float64
Score is 0.98855437


# Submission

In [18]:
label_dict = {
   0 : 'HDMI',
   1 : 'USB_Mini',
   2 : 'Dock',
   3 : 'Lightning',
   4 : 'USB_Micro_B',
   5 : 'Mini_DisplayPort',
   6 : 'RJ_45',
   7 : 'USB_Type_A',
   8 : 'USB_Type_C',
   9 : 'USB_Micro_B_3.1',
  10 : 'USB_Type_B',
  11 : 'Lightning_T',
  12 : 'USB_Micro_B_W',
  13 : 'VGA',
  14 : 'DisplayPort'
}

In [19]:
import scipy.stats as stats

#print(stacking_sub)
results = stats.mode(stacking_sub, axis=0).mode[0]
print(len(results))

# class num to class name
targets = []
for i in range(len(results)):
    targets.append(label_dict[results[i]])
sub['class'] = targets
print(len(targets))

test = pd.read_csv('../input/test_with_fold.csv')
sub['filename'] = test['filename']

sub_name = '_'.join(versions)
sub[['filename', 'class']].to_csv(f'../output/ensemble/{pref}_stacking_{sub_name}.csv', index=False)

2200
2200
